In [10]:
#import bibliotek

import pandas as pd
import numpy as np

In [11]:
#importyje dane filmowe, ustawiam low_memory na False ponieważ rzuca błędem o mieszanym typie danych
data=pd.read_csv('movies_metadata.csv', low_memory=False)

In [ ]:
Wyświetlam inforacje o bazie danych

In [12]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                            overview  ... release_date  \
0  Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1  When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2  A family wedding reignites the ancient feud be...  ...   1995-12-22   
3  Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4  Just when George Banks has recovered from his ...  ...   1995-02-10   

       revenue runtime                                   spoken_languages  \
0  373554033.0    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
2          0.0   101.0           [{'iso_639_1': 'en', 'name': 'English'}]   
3   81452156.0   127.0           [{'iso_639_1': 'en', 'name': 'English'}]   
4   76578911.0   106.0           [{'iso_639_1': 'en', 'name': 'English'}]   

     status                                            tagline  \
0  Released                                                NaN   
1  Released          Roll the dice and unleash the excitement!   
2  Released  Still Yelling. Still Fighting. Still Ready for...   
3  Released  Friends are the people who let you be yourself...   
4  Released  Just When His World Is Back To Normal... He's ...   

                         title  video vote_average vote_count  
0                    Toy Story  False          7.7     5415.0  
1                      Jumanji  False          6.9     2413.0  
2             Grumpier Old Men  False          6.5       92.0  
3            Waiting to Exhale  False          6.1       34.0  
4  Father of the Bride Part II  False          5.7      173.0  

[5 rows x 24 columns]

In [26]:
#v_count=vote_count  
#v_avg=vote_average
v_count_min                       #to minimalna liczba głosów wymagana do umieszczenia na wykresie m
v_avg_all                         #to średnia ocen wszystkich filmów c

#wzór na średnią ważoną
#weight_rating=(v_count/(v_count+v_count_min)*v_avg_all+(v_count_min/(v_count_min+v_count)*v_avg_all

5.618207215133889

In [14]:
v_avg_all=data['vote_average'].mean() #średnia dla kolumny vote average
v_avg_all

5.618207215133889

In [15]:
v_count_min=data['vote_count'].quantile(0.90) #minimalna liczba wymaganych ocen. zastosowałam kwantyl 90
v_count_min

160.0

In [16]:
new_movies = data.copy().loc[data['vote_count'] >= v_count_min] # filtruje wszystkie kwalifikujące się filmy do nowej tabeli data
new_movies.shape

(4555, 24)

In [17]:
data.shape #wniosek ~ 10% filmów z liczbą głosów przekraczającą 160 się kwalifikuje 

(45466, 24)

In [32]:
#Funkcja obliczająca ważoną ocenę każdego filmu

def weight_rating(x, v_count_min = v_count_min, v_avg_all = v_avg_all):
    v_count = x['vote_count']  
    v_avg = x['vote_average']
    return (v_count/(v_count+v_count_min)*v_avg)+(v_count_min/(v_count_min+v_count)*v_avg_all)

In [33]:
#definiuję nową cechę „score” i obliczam jej wartość za pomocą funkcji jw

new_movies['score'] = new_movies.apply(weight_rating, axis=1) 

In [34]:
#sortowanie wg scoringu, ascending ustawiono na false żeby podana lista była rosnąco
new_movies = new_movies.sort_values('score', ascending=False)

In [35]:
#wyświetlam 20 najwyzej oceniach filmów po scoringu
new_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

title  vote_count  vote_average     score
314           The Shawshank Redemption      8358.0           8.5  8.445869
834                      The Godfather      6024.0           8.5  8.425439
10309      Dilwale Dulhania Le Jayenge       661.0           9.1  8.421453
12481                  The Dark Knight     12269.0           8.3  8.265477
2843                        Fight Club      9678.0           8.3  8.256385
292                       Pulp Fiction      8670.0           8.3  8.251406
522                   Schindler's List      4436.0           8.3  8.206639
23673                         Whiplash      4376.0           8.3  8.205404
5481                     Spirited Away      3968.0           8.3  8.196055
2211                 Life Is Beautiful      3643.0           8.3  8.187171
1178            The Godfather: Part II      3418.0           8.3  8.180076
1152   One Flew Over the Cuckoo's Nest      3001.0           8.3  8.164256
351                       Forrest Gump      8147.0           8.2  8.150272
1154           The Empire Strikes Back      5998.0           8.2  8.132919
1176                            Psycho      2405.0           8.3  8.132715
18465                 The Intouchables      5410.0           8.2  8.125837
40251                       Your Name.      1030.0           8.5  8.112532
289             Leon: The Professional      4293.0           8.2  8.107234
3030                    The Green Mile      4166.0           8.2  8.104511
1170                        GoodFellas      3211.0           8.2  8.077459